In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
type(mnist)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
single_image = mnist.train.images[2].reshape(28,28)

In [ ]:
plt.imshow(single_image,cmap='gist_gray')

In [ ]:
x = tf.placeholder(tf.float32,[None,784])
w = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [ ]:
y = tf.matmul(x,w) + b

In [ ]:
y_true = tf.placeholder(tf.float32,[None,10])

In [ ]:
loss= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.5)
train = optimizer.minimize(loss)

In [ ]:
init =tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for step in range(1000):
        batch_x , batch_y = mnist.train.next_batch(100)
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y})
        correct_predict = tf.equal(tf.argmax(y,1),tf.argmax(y_true,1))
        acc = tf.reduce_mean(tf.cast(correct_predict,tf.float32))
    print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels}))

### Using CNN

In [ ]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)

In [ ]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

In [ ]:
def conv2d(x,w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME')

In [ ]:
def max_pool_2by2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [ ]:
def convolution_layer(input_x,shape):
    w = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,w)+b)

In [ ]:
def normal_full_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    w = init_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,w )+ b

In [ ]:
x = tf.placeholder(tf.float32,shape=[None,784])

In [ ]:
y_true = tf.placeholder(tf.float32,shape=[None,10])

In [ ]:
x_image = tf.reshape(x,[-1,28,28,1])

In [ ]:
convo1 = convolution_layer(x_image,shape=[5,5,1,32])
convo1_pooling = max_pool_2by2(convo1)

In [ ]:
convo2 = convolution_layer(convo1_pooling,shape=[5,5,32,64])
convo2_pooling = max_pool_2by2(convo2)

In [ ]:
convo2_flat = tf.reshape(convo2_pooling,(-1,7*7*64))
full_layer_one = tf.nn.relu(normal_full_layer(convo2_flat,1024))

In [ ]:
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [ ]:
y_pred = normal_full_layer(full_one_dropout,10)

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

In [ ]:
optimizer= tf.train.AdamOptimizer(learning_rate=0.001)
train=optimizer.minimize(cross_entropy)

In [ ]:
init = tf.global_variables_initializer()

In [29]:
with tf.Session() as sess:
    sess.run(init)
    batches = 400
    for i in range(batches):
        x_batch,y_batch = mnist.test.next_batch(50)
        sess.run(train,feed_dict={x:x_batch,y_true:y_batch,hold_prob:0.5})
        if i%100 ==0:
            match = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(match,tf.float32))
            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))

0.1074
0.9621
0.972
0.9864


### Keras

In [32]:
import keras
from keras import models,layers

In [33]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [34]:
from keras.datasets import mnist
from keras.utils import to_categorical
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [35]:
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 122s 2ms/step - loss: 0.1777 - acc: 0.9449
Epoch 2/5
60000/60000 [==============================] - 143s 2ms/step - loss: 0.0486 - acc: 0.9848
Epoch 3/5
60000/60000 [==============================] - 134s 2ms/step - loss: 0.0329 - acc: 0.9900
Epoch 4/5
60000/60000 [==============================] - 93s 2ms/step - loss: 0.0253 - acc: 0.9921
Epoch 5/5
60000/60000 [==============================] - 91s 2ms/step - loss: 0.0198 - acc: 0.9939


In [36]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 8s 762us/step


0.9916